In [ ]:
from tkinter import *
from tkinter import filedialog

# A function to allow the user to select the folder contianing the subfolders of images.
# Function inputs args: None. 
# Function output 1: The path of the folder selected by the user. 
def folder_selection_dialog():
    root = Tk()
    root.title('Please select the directory containing the folders fo simulation data.')
    root.filename = filedialog.askdirectory(initialdir="/", title="Select A Folder")
    simulation_directory = root.filename
    root.destroy()

    return simulation_directory